In [1]:
from google.cloud import storage
import os
import glob
import pandas as pd
import psycopg2

In [2]:
import configs
import fenestraTask

In [4]:
fenestraTask.db_operation.__doc__

"\n    Perform various database operations based on the specified operation type.\n\n    This function establishes a connection to a PostgreSQL database using provided\n    connection parameters and executes an operation based on the specified operation type.\n    The operations include listing tables, creating a new table, dropping tables,\n    showing top rows from a table, and executing a custom query.\n\n    Parameters:\n    operation_type (str): The type of database operation to perform. \n                          Valid options include 'list_tables', 'create_table', \n                          'drop_tables', 'show_top_rows', 'custom_query'.\n    connection_params (dict): A dictionary containing database connection parameters \n                              such as 'host', 'database', 'user', 'password'.\n    table_name (str, optional): Name of the table to be used or created. \n                                Required for 'create_table', 'drop_tables', 'show_top_rows'.\n    query

In [17]:
connection_params = configs.DB_CONFIG

In [25]:
service_account_key_path = configs.TASK_CONFIG["service_account_key_path"]
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key_path
bucket_name = configs.TASK_CONFIG["bucket_name"]
local_download_path = configs.TASK_CONFIG["local_download_path"]
downloaded_files_record = configs.TASK_CONFIG["downloaded_files_record"]
processed_files_record = configs.TASK_CONFIG["processed_files_record"]

In [21]:
table_name = 'ads_data_imp'

In [7]:
fenestraTask.db_operation('create_table', connection_params=connection_params, table_name="ads_data_imp")

In [20]:
fenestraTask.download_new_files(bucket_name, local_download_path, downloaded_files_record)

All files from the bucket have already been downloaded.


In [26]:
fenestraTask.process_and_ingest_files(local_download_path, processed_files_record, table_name)

All files have been ingested.


In [27]:
fenestraTask.db_operation('list_tables',connection_params)

ads_data_imp


In [18]:
tables_to_drop = ["ads_data_imps_2","ads_data"]
fenestraTask.db_operation('drop_tables', connection_params, table_list=tables_to_drop)

Table 'ads_data_imps_2' dropped.
Table 'ads_data' dropped.


# Total Number of Rows

In [22]:
fenestraTask.db_operation("custom_query",connection_params, query="SELECT COUNT(*) FROM ads_data_imp")

(8318525,)


# Columns

In [28]:
fenestraTask.db_operation("custom_query",connection_params, query="SELECT column_name FROM information_schema.columns WHERE table_schema = 'public' AND table_name = 'ads_data_imp'")

('time',)
('advertiserid',)
('orderid',)
('lineitemid',)
('creativeid',)
('creativeversion',)
('creativesize',)
('adunitid',)
('domain',)
('countryid',)
('regionid',)
('metroid',)
('cityid',)
('browserid',)
('osid',)
('osversion',)
('timeusec2',)
('keypart',)
('product',)
('requestedadunitsizes',)
('bandwidthgroupid',)
('mobiledevice',)
('iscompanion',)
('devicecategory',)
('activevieweligibleimpression',)
('mobilecarrier',)
('estimatedbackfillrevenue',)
('gfpcontentid',)
('postalcodeid',)
('bandwidthid',)
('audiencesegmentids',)
('mobilecapability',)
('publisherprovidedid',)
('videoposition',)
('podposition',)
('videofallbackposition',)
('isinterstitial',)
('eventtimeusec2',)
('eventkeypart',)
('yieldgroupcompanyid',)
('requestlanguage',)
('dealid',)
('sellerreserveprice',)
('dealtype',)
('adxaccountid',)
('buyer',)
('advertiser',)
('anonymous',)
('impressionid',)


# a. How many records are there per day and per hour?

In [30]:
fenestraTask.db_operation("custom_query",connection_params, query="SELECT LEFT(Time, 10) AS Date, SUBSTRING(Time, 12, 2) AS Hour, COUNT(*) AS RecordCount FROM ads_data_imp GROUP BY Date, Hour ORDER BY Date, Hour")

('2019-09-01', '07', 1922)
('2019-09-01', '08', 619719)
('2019-09-01', '09', 546879)
('2019-09-01', '10', 450611)
('2019-09-01', '11', 391433)
('2019-09-01', '12', 361572)
('2019-09-01', '13', 377933)
('2019-09-01', '14', 349846)
('2019-09-01', '15', 343789)
('2019-09-01', '16', 355513)
('2019-09-01', '17', 355473)
('2019-09-01', '18', 351378)
('2019-09-01', '19', 373065)
('2019-09-01', '20', 409407)
('2019-09-01', '21', 432227)
('2019-09-01', '22', 410626)
('2019-09-01', '23', 302934)
('2019-09-02', '00', 195228)
('2019-09-02', '01', 147194)
('2019-09-02', '02', 134168)
('2019-09-02', '03', 137303)
('2019-09-02', '04', 145686)
('2019-09-02', '05', 204409)
('2019-09-02', '06', 381237)
('2019-09-02', '07', 538973)


# b. What is the total of the EstimatedBackFillRevenue field per day and per hour

In [31]:
fenestraTask.db_operation("custom_query",connection_params, query="SELECT LEFT(Time, 10) AS Date, SUBSTRING(Time, 12, 2) AS Hour, SUM(EstimatedBackfillRevenue) AS TotalRevenue FROM ads_data_imp GROUP BY Date, Hour ORDER BY Date, Hour")

('2019-09-01', '07', Decimal('0.8335734388884032191784'))
('2019-09-01', '08', Decimal('202.2057374181697037894383'))
('2019-09-01', '09', Decimal('186.4349177893931070293848'))
('2019-09-01', '10', Decimal('159.1024052623410633852715'))
('2019-09-01', '11', Decimal('143.5612254276205989843509'))
('2019-09-01', '12', Decimal('135.6108092889890548586095'))
('2019-09-01', '13', Decimal('144.7465090085866646002467'))
('2019-09-01', '14', Decimal('138.3017635600974612789538'))
('2019-09-01', '15', Decimal('132.7623508021526204241631'))
('2019-09-01', '16', Decimal('134.1169707038224484628050'))
('2019-09-01', '17', Decimal('126.0064579681296510230156'))
('2019-09-01', '18', Decimal('122.3418028032400099974698'))
('2019-09-01', '19', Decimal('122.2833037258964397361316'))
('2019-09-01', '20', Decimal('124.2178134490790983583709'))
('2019-09-01', '21', Decimal('121.2184865853885268430332'))
('2019-09-01', '22', Decimal('112.4167751439715904660451'))
('2019-09-01', '23', Decimal('91.201403372

# c. How many records and what is the total of the EstimatedBackFillRevenue field per Buyer?

In [34]:
fenestraTask.db_operation("custom_query",connection_params, query="SELECT Buyer, COUNT(*) AS RecordCount, SUM(EstimatedBackfillRevenue) AS TotalRevenue FROM ads_data_imp GROUP BY Buyer ORDER BY TotalRevenue DESC")

('Google AdWords', 4415341, Decimal('1098.0846796178389793280320'))
('PubMatic (EB)', 280785, Decimal('210.677439126704056029742'))
('Rubicon (EB)', 345977, Decimal('181.780219457398453006247'))
('Index Exchange (EB)', 418144, Decimal('166.4635628530060737107575'))
('DoubleClick Bid Manager (Europe)', 200167, Decimal('134.0452139171883589668991'))
('Criteo (UK)', 273454, Decimal('132.3224766878745176922000'))
('OpenX (EB)', 302191, Decimal('131.8813076154407354886230'))
('TripleLift (EB)', 153778, Decimal('109.400828281159895057127'))
('Amazon EU', 119702, Decimal('59.1615141741159075665922'))
('Google: DCLK Bid Manager', 56352, Decimal('54.7718270566631802813959'))
('Sovrn (EB)', 44288, Decimal('32.6094423612195658085250'))
('DBM AUNZ', 23232, Decimal('28.9617383300555775115638'))
('RhythmOne (EB)', 11449, Decimal('28.73083726647599044411'))
('Infectious Media w/DBM', 38281, Decimal('27.668735758394800143359'))
('GroupM PBU (MediaCom UK IE)', 2871, Decimal('21.877614687266169023211'))

AttributeError: 'NoneType' object has no attribute 'to_csv'

# d. List the unique Device Categories by Advertiser

In [35]:
fenestraTask.db_operation("custom_query",connection_params, query="SELECT Advertiser, ARRAY_AGG(DISTINCT DeviceCategory) AS UniqueDeviceCategories FROM ads_data_imp GROUP BY Advertiser ORDER BY Advertiser")

('렌딧', ['Desktop', 'Tablet'])
('세스코', ['Smartphone', 'Tablet'])
('야놀자', ['Smartphone'])
('에듀윌', ['Desktop', 'Smartphone'])
('칸투칸', ['Desktop', 'Smartphone', 'Tablet'])
('파고다 교육그룹', ['Smartphone'])
('내일투어', ['Desktop'])
('랭킹닭컴', ['Desktop', 'Tablet'])
('아이비젼', ['Desktop'])
('아이소이', ['Desktop'])
('주식회사 퀄슨', ['Desktop', 'Smartphone'])
('주식회사 심야모임', ['Smartphone'])
('주식회사 마크마운트', ['Desktop'])
('ヴィエリス', ['Smartphone'])
('대성마이맥', ['Desktop'])
('박문각그룹', ['Desktop'])
('배달의민족', ['Smartphone'])
('서울문화사', ['Smartphone'])
('에코마케팅', ['Desktop'])
('오늘안치과', ['Smartphone'])
('エコスタイル', ['Desktop'])
('한성아카데미', ['Tablet'])
('บริษัท เลนโซ่ วีล จำกัด', ['Desktop', 'Smartphone', 'Tablet'])
('บริษัท นารายณ์พร็อพเพอตี้ จำกัด', ['Desktop', 'Smartphone'])
('セキスイハイム', ['Desktop', 'Smartphone'])
('ベンチャーバンク', ['Desktop'])
('레진엔터테인먼트', ['Desktop', 'Tablet'])
('서울게임아카데미', ['Desktop', 'Smartphone'])
('아이에스이커머스', ['Smartphone'])
('이지드림대부중개', ['Smartphone'])
('1000Bulbs.com', ['Desktop', 'Smartphone', 'Tablet'])
('1001

('Certified Financial Planner Board of Standards', ['Desktop', 'Smartphone'])
('Cetmen Magazalari', ['Smartphone', 'Tablet'])
('Cewe Stiftung & Co. KGaA', ['Desktop', 'Smartphone', 'Tablet'])
('CFA Institute', ['Desktop', 'Tablet'])
('Chairish, Inc.', ['Desktop', 'Smartphone', 'Tablet'])
('Chair King Backyard Store', ['Smartphone', 'Tablet'])
('Chal-Tec', ['Desktop', 'Smartphone', 'Tablet'])
('Champion Windows', ['Smartphone', 'Tablet'])
('Channel Breeze', ['Smartphone', 'Tablet'])
('Chapman Automotive Group', ['Smartphone', 'Tablet'])
('CharityUSA.com, LLC', ['Desktop', 'Smartphone'])
('Charles Barker Automotive', ['Smartphone'])
('Charles & Colvard', ['Desktop', 'Smartphone', 'Tablet'])
('Charles Schwab', ['Desktop', 'Smartphone', 'Tablet'])
('Charles Tyrwhitt', ['Desktop', 'Smartphone', 'Tablet'])
('Charlotte Tilbury', ['Desktop', 'Smartphone', 'Tablet'])
('Charter Communications', ['Desktop', 'Smartphone', 'Tablet'])
('Chartered Accountants Australia & New Zealand', ['Desktop', 'Sm

('Isha Foundation', ['Desktop', 'Smartphone', 'Tablet'])
('Islamic Relief', ['Desktop', 'Smartphone', 'Tablet'])
('Isoroot', ['Smartphone'])
('Israel Blessing', ['Desktop', 'Smartphone', 'Tablet'])
('Israeli Government', ['Connected TV', 'Desktop', 'Smartphone', 'Tablet'])
('Isrotel - Hotels Chain', ['Desktop'])
('issta lines', ['Desktop', 'Tablet'])
('Istanbul Bilisim', ['Desktop', 'Smartphone'])
('Istituto Europeo di Design', ['Desktop', 'Smartphone'])
('IST Media', ['Smartphone'])
('iStore', ['Desktop', 'Smartphone', 'Tablet'])
('Isuzu', ['Connected TV', 'Desktop', 'Smartphone', 'Tablet'])
('Itaka', ['Smartphone'])
('Itatiaia', ['Desktop', 'Smartphone', 'Tablet'])
('Itaúsa', ['Desktop', 'Smartphone', 'Tablet'])
('ITC', ['Desktop', 'Smartphone'])
('IT Channel Pty Ltd.', ['Desktop', 'Smartphone'])
('Ithuba Holdings Proprietary', ['Desktop', 'Smartphone', 'Tablet'])
('iTrust Financial Advisors', ['Desktop'])
('iTutorGroup', ['Desktop', 'Smartphone', 'Tablet'])
('ITW Global Brands', ['D

('RSM International', ['Desktop', 'Smartphone'])
('Ruby Tuesday', ['Desktop', 'Smartphone', 'Tablet'])
('ruedelafete.com', ['Desktop'])
('Ruggable', ['Desktop', 'Smartphone', 'Tablet'])
('Rugs Direct', ['Desktop', 'Smartphone', 'Tablet'])
('RugsUSA', ['Desktop', 'Smartphone'])
('Ruibo', ['Desktop', 'Smartphone', 'Tablet'])
('Russia Today', ['Smartphone'])
('Russums', ['Desktop'])
('Rustica Home and Hardware', ['Smartphone', 'Tablet'])
('Rustomjee', ['Desktop', 'Smartphone'])
('Rutgers University', ['Smartphone'])
('Ruths Chris Steak House', ['Smartphone'])
('rvusa', ['Smartphone'])
('RWE Group', ['Desktop', 'Smartphone', 'Tablet'])
('RW Lynch', ['Smartphone'])
('Ryanair Holdings', ['Desktop', 'Smartphone', 'Tablet'])
('Ryder System', ['Desktop', 'Smartphone', 'Tablet'])
('RZD Group', ['Smartphone'])
('Saatva', ['Desktop', 'Smartphone', 'Tablet'])
('Saat Ve Saat Magazacilik', ['Smartphone'])
('Sabanci Holding', ['Desktop', 'Smartphone', 'Tablet'])
('Sabon Holdings SRL', ['Smartphone'])


# e. How many duplicate rows were there?

In [40]:
fenestraTask.db_operation("custom_query",connection_params, query="SELECT SUM(Count) - COUNT(*) AS TotalDuplicates FROM (SELECT COUNT(*) AS Count FROM ads_data_imp GROUP BY time, advertiserid, orderid, lineitemid, creativeid, creativeversion, creativesize, adunitid,domain, countryid, regionid, metroid, cityid, browserid, osid, osversion, timeusec2,keypart, product, requestedadunitsizes, bandwidthgroupid, mobiledevice, iscompanion,devicecategory, activevieweligibleimpression, mobilecarrier, estimatedbackfillrevenue, gfpcontentid,postalcodeid, bandwidthid, audiencesegmentids, mobilecapability, publisherprovidedid, videoposition,podposition, videofallbackposition, isinterstitial, eventtimeusec2, eventkeypart, yieldgroupcompanyid,requestlanguage, dealid, sellerreserveprice, dealtype, adxaccountid, buyer, advertiser, anonymous, impressionid HAVING COUNT(*) > 1) AS DuplicateGroups")

(Decimal('5999'),)


- *5999, duplicates rows.*

# Procedure to remove all duplicates.

- SELECT MIN(ctid), ctid FROM ads_data_imp GROUP BY "all_columns_except_ctid" HAVING COUNT(*) > 1;
- DELETE FROM ads_data_imp WHERE ctid NOT IN ( SELECT MIN(ctid) FROM ads_data_imp GROUP BY "all_columns_except_ctid");


----------